# Notebook 05: Extração Detalhada de 19 Campos com Gemini

Este notebook foca em realizar uma extração de dados altamente específica e estruturada a partir dos textos processados (resumos e títulos de artigos/patentes). Utilizaremos o Google Gemini com um prompt detalhado para preencher 19 campos pré-definidos, visando criar um dataset rico para análises futuras e para o sistema de recomendação de formulações de tintas nanotecnológicas.

## 1. Configuração Inicial e Carregamento de Dados

In [28]:
import json
import pandas as pd
import google.generativeai as genai
import time
import os
from pathlib import Path
from tqdm.auto import tqdm
import random

tqdm.pandas()

In [29]:
# Carregar configurações do sistema
config_path = Path('config_sistema.json')
if config_path.exists():
    with open(config_path, 'r', encoding='utf-8') as f:
        config_sistema = json.load(f)
else:
    print("Arquivo de configuração 'config_sistema.json' não encontrado. Execute o notebook 01 primeiro.")
    config_sistema = {}

# Caminhos (baseados no config_sistema.json)
API_KEY_FILE = Path(config_sistema.get('API_KEY_PATH', 'api_key.txt'))
PROCESSED_DATA_PATH = Path(config_sistema.get('PROCESSED_DATA_PATH', '../data/processed/'))

# Arquivo de entrada (saída do notebook 03 - análise regex)
GEMINI_ANALYSIS_INPUT_FILE = PROCESSED_DATA_PATH / 'dataset_with_regex_analysis.csv'

# Saídas deste notebook
EXTRACTED_19_FIELDS_OUTPUT_FILE = PROCESSED_DATA_PATH / 'dataset_with_19_fields.csv'
EXTRACTED_19_FIELDS_SUMMARY_FILE = PROCESSED_DATA_PATH / 'extraction_19_fields_summary.json'

# Criar diretório de dados processados se não existir
PROCESSED_DATA_PATH.mkdir(parents=True, exist_ok=True)

print(f"Caminho do arquivo da API Key: {API_KEY_FILE}")
print(f"Caminho dos dados processados: {PROCESSED_DATA_PATH}")
print(f"Arquivo de entrada (análise regex anterior): {GEMINI_ANALYSIS_INPUT_FILE}")
print(f"Arquivo de saída (19 campos extraídos): {EXTRACTED_19_FIELDS_OUTPUT_FILE}")
print(f"Arquivo de sumário da extração (19 campos): {EXTRACTED_19_FIELDS_SUMMARY_FILE}")

Caminho do arquivo da API Key: api_key.txt
Caminho dos dados processados: ../data/processed
Arquivo de entrada (análise regex anterior): ../data/processed/dataset_with_regex_analysis.csv
Arquivo de saída (19 campos extraídos): ../data/processed/dataset_with_19_fields.csv
Arquivo de sumário da extração (19 campos): ../data/processed/extraction_19_fields_summary.json


In [30]:
# Configurar API Key do Google Gemini
try:
    with open(API_KEY_FILE, 'r') as f:
        api_key = f.read().strip()
    genai.configure(api_key=api_key)
    print("API Key do Google Gemini configurada com sucesso.")
except FileNotFoundError:
    print(f"Arquivo da API Key '{API_KEY_FILE}' não encontrado. Verifique o caminho e o arquivo.")
    api_key = None
except Exception as e:
    print(f"Erro ao configurar a API Key: {e}")
    api_key = None

API Key do Google Gemini configurada com sucesso.


In [31]:
# Carregar dados da análise Gemini anterior (saída do notebook 04)
if GEMINI_ANALYSIS_INPUT_FILE.exists():
    df_input = pd.read_csv(GEMINI_ANALYSIS_INPUT_FILE)
    print(f"Dataset da análise Gemini anterior carregado: {len(df_input)} registros.")
    print("Colunas disponíveis:", df_input.columns.tolist())
    print(df_input.head())
    if 'Abstract_to_analyze' not in df_input.columns:
        print("ALERTA: Coluna 'Abstract_to_analyze' não encontrada. Esta coluna é essencial para a extração." )       # Tentar criar a coluna 'Abstract_to_analyze' usando os nomes corretos das colunas
        if 'Article Title' in df_input.columns and 'Abstract' in df_input.columns:
            df_input['Abstract_to_analyze'] = df_input['Article Title'].fillna('') + " " + df_input['Abstract'].fillna('')
            print("Coluna 'Abstract_to_analyze' criada a partir da concatenação de 'Article Title' e 'Abstract'.")
        elif 'TI' in df_input.columns and 'AB' in df_input.columns:
            df_input['Abstract_to_analyze'] = df_input['TI'].fillna('') + " " + df_input['AB'].fillna('')
            print("Coluna 'Abstract_to_analyze' criada a partir da concatenação de 'TI' e 'AB'.")
        else:
            print("Não foi possível criar 'Abstract_to_analyze'. A extração pode falhar ou ser de baixa qualidade.")
            df_input['Abstract_to_analyze'] = "" # Coluna vazia para evitar erros, mas resultará em má extração
else:
    print(f"Arquivo '{GEMINI_ANALYSIS_INPUT_FILE}' não encontrado. Execute o notebook 04 primeiro.")
    df_input = pd.DataFrame() # DataFrame vazio para evitar erros subsequentes

Dataset da análise Gemini anterior carregado: 9046 registros.
Colunas disponíveis: ['Publication Type', 'Authors', 'Book Authors', 'Book Editors', 'Book Group Authors', 'Author Full Names', 'Book Author Full Names', 'Group Authors', 'Article Title', 'Source Title', 'Book Series Title', 'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title', 'Conference Date', 'Conference Location', 'Conference Sponsor', 'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract', 'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses', 'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred', 'Funding Text', 'Cited References', 'Cited Reference Count', 'Times Cited, WoS Core', 'Times Cited, All Databases', '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher', 'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN', 'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date', 'Publication Year', 'Volume', 'Issue', 'Part Number', 'S

/tmp/ipykernel_122208/2015955678.py:3: DtypeWarning: Columns (7,47,50,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  df_input = pd.read_csv(GEMINI_ANALYSIS_INPUT_FILE)


## 2. Definição dos 19 Campos para Extração e Configuração da API Gemini

Os 19 campos a serem extraídos são:
1.  `id_documento`: Identificador único do documento (ex: DOI, número de patente, ou ID interno).
2.  `titulo`: Título completo do documento.
3.  `ano`: Ano de publicação.
4.  `tipo_documento`: Tipo de documento (ex: Artigo Científico, Patente, Revisão, Conferência).
5.  `autores`: Lista dos nomes dos autores.
6.  `afiliacoes_autores`: Lista das afiliações dos autores.
7.  `revista_patente_info`: Nome da revista, conferência ou informações da patente (escritório, número).
8.  `pais_origem`: País de publicação ou origem da patente.
9.  `palavras_chave_documento`: Lista das palavras-chave fornecidas pelo documento.
10. `nanomateriais_citados`: Lista dos nanomateriais específicos investigados ou desenvolvidos.
11. `metodo_sintese_fabricacao_detalhado`: Descrição detalhada dos métodos de síntese dos nanomateriais ou fabricação do produto/tinta.
12. `precursores_reagentes`: Lista dos principais precursores, reagentes ou matérias-primas utilizadas.
13. `parametros_processo`: Parâmetros críticos do processo (ex: temperatura, tempo, pressão, concentração, pH).
14. `tecnicas_caracterizacao`: Lista das técnicas de caracterização empregadas para analisar os materiais ou propriedades.
15. `propriedades_resultados_especificos`: Propriedades chave investigadas e resultados quantitativos específicos (ex: "eficiência de 95%", "tamanho da partícula de 20-50 nm", "condutividade de 150 S/cm").
16. `aplicacao_primaria`: Principal aplicação visada ou testada para o nanomaterial ou produto.
17. `mecanismos_acao_explicados`: Breve descrição de quaisquer mecanismos de ação, funcionamento ou fenômenos explicados.
18. `vantagens_inovacao_declarada`: Vantagens ou inovações destacadas no estudo.
19. `desafios_limitacoes_futuro`: Desafios, limitações identificadas ou trabalhos futuros sugeridos.

Se uma informação não estiver explicitamente presente no texto, o campo correspondente no JSON deve ser `null`, uma string vazia (`""`), ou uma lista vazia (`[]`), conforme apropriado para o tipo de dado do campo.

In [32]:
# Parâmetros da API Gemini
GEMINI_MODEL_NAME = config_sistema.get('GEMINI_MODEL_NAME_DETAILED_EXTRACTION', 'gemini-2.0-pro')
MAX_RETRIES = config_sistema.get('MAX_RETRIES_DETAILED_EXTRACTION', 3)
RETRY_DELAY = config_sistema.get('RETRY_DELAY_DETAILED_EXTRACTION', 10)  # segundos, maior para prompts complexos
REQUEST_TIMEOUT = config_sistema.get('REQUEST_TIMEOUT_DETAILED_EXTRACTION', 300) # segundos, maior para prompts complexos
RATE_LIMIT_DELAY = config_sistema.get('RATE_LIMIT_DELAY_DETAILED_EXTRACTION', 2) # segundos entre requisições

generation_config = {
    "temperature": config_sistema.get('GEMINI_TEMPERATURE_DETAILED_EXTRACTION', 0.5),
    "top_p": config_sistema.get('GEMINI_TOP_P_DETAILED_EXTRACTION', 0.95),
    "top_k": config_sistema.get('GEMINI_TOP_K_DETAILED_EXTRACTION', 40),
    "max_output_tokens": config_sistema.get('GEMINI_MAX_OUTPUT_TOKENS_DETAILED_EXTRACTION', 4096), # Aumentado para JSON complexo
    "response_mime_type": "application/json",
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

if api_key:
    model_19_fields = genai.GenerativeModel(
        model_name=GEMINI_MODEL_NAME,
        generation_config=generation_config,
        safety_settings=safety_settings
    )
    print(f"Modelo Gemini para extração dos 19 campos ({GEMINI_MODEL_NAME}) inicializado.")
else:
    model_19_fields = None
    print("Modelo Gemini para extração dos 19 campos não pôde ser inicializado (API Key ausente).")

# Lista dos 19 campos para referência no código
FIELDS_TO_EXTRACT = [
    "id_documento", "titulo", "ano", "tipo_documento", "autores", 
    "afiliacoes_autores", "revista_patente_info", "pais_origem", 
    "palavras_chave_documento", "nanomateriais_citados", 
    "metodo_sintese_fabricacao_detalhado", "precursores_reagentes", 
    "parametros_processo", "tecnicas_caracterizacao", 
    "propriedades_resultados_especificos", "aplicacao_primaria", 
    "mecanismos_acao_explicados", "vantagens_inovacao_declarada", 
    "desafios_limitacoes_futuro"
]

Modelo Gemini para extração dos 19 campos (gemini-2.0-pro) inicializado.


In [33]:
def generate_extraction_prompt(text_content):
    """
    Gera o prompt para a API Gemini solicitando a extração dos 19 campos.
    """
    prompt = f"""
    Analise o seguinte texto (que pode ser um resumo, título ou combinação de partes de um artigo científico ou patente) sobre nanotecnologia, nanomateriais ou tintas funcionais.
    Sua tarefa é extrair informações detalhadas e estruturadas correspondentes aos 19 campos listados abaixo.
    Retorne a sua resposta EXCLUSIVAMENTE como um único objeto JSON. Não inclua explicações antes ou depois do JSON.
    O objeto JSON deve conter as seguintes chaves (exatamente como escritas):
    1.  `id_documento`: (String) Identificador único do documento (ex: DOI, número de patente, ID interno se mencionado no texto). Se não encontrado, use null.
    2.  `titulo`: (String) Título completo do documento, conforme presente no texto. Se não encontrado, use null.
    3.  `ano`: (Integer or String) Ano de publicação. Extraia apenas o ano. Se não encontrado, use null.
    4.  `tipo_documento`: (String) Tipo de documento (ex: "Artigo Científico", "Patente", "Revisão", "Artigo de Conferência"). Inferir do contexto se possível. Se não encontrado, use null.
    5.  `autores`: (List of Strings) Lista dos nomes dos autores. Ex: ["Silva, J.", "Santos, M."]. Se não encontrado, use [].
    6.  `afiliacoes_autores`: (List of Strings) Lista das afiliações dos autores. Ex: ["Universidade X, Brasil", "Instituto Y, EUA"]. Se não encontrado, use [].
    7.  `revista_patente_info`: (String) Nome da revista, anais de conferência, ou informações da patente (ex: "Journal of Nanomaterials", "US Patent Office 1234567"). Se não encontrado, use null.
    8.  `pais_origem`: (String) País de publicação ou de origem da patente, se mencionado. Se não encontrado, use null.
    9.  `palavras_chave_documento`: (List of Strings) Lista das palavras-chave fornecidas no documento. Se não encontrado, use [].
    10. `nanomateriais_citados`: (List of Strings) Lista dos nanomateriais específicos investigados ou desenvolvidos (ex: ["nanopartículas de TiO2", "nanotubos de carbono", "grafeno"]). Se não encontrado, use [].
    11. `metodo_sintese_fabricacao_detalhado`: (String) Descrição concisa, mas informativa, dos métodos de síntese dos nanomateriais ou fabricação do produto/tinta. Se não encontrado, use null.
    12. `precursores_reagentes`: (List of Strings) Lista dos principais precursores, reagentes ou matérias-primas utilizadas. Se não encontrado, use [].
    13. `parametros_processo`: (List of Strings) Parâmetros críticos do processo mencionados (ex: ["Temperatura: 150°C", "Tempo de reação: 24h", "pH: 7.0"]). Se não encontrado, use [].
    14. `tecnicas_caracterizacao`: (List of Strings) Lista das técnicas de caracterização empregadas (ex: ["MEV", "TEM", "DRX", "FTIR", "UV-Vis"]). Se não encontrado, use [].
    15. `propriedades_resultados_especificos`: (List of Strings) Propriedades chave investigadas e resultados quantitativos específicos (ex: ["Eficiência de remoção de poluente: 95%", "Tamanho médio de partícula: 30 nm", "Resistividade: 0.01 Ohm.cm"]). Se não encontrado, use [].
    16. `aplicacao_primaria`: (String) Principal aplicação visada, testada ou proposta para o nanomaterial ou produto/tinta. Se não encontrado, use null.
    17. `mecanismos_acao_explicados`: (String) Breve descrição de quaisquer mecanismos de ação, funcionamento ou fenômenos científicos explicados no texto. Se não encontrado, use null.
    18. `vantagens_inovacao_declarada`: (String) Vantagens, benefícios ou inovações destacadas no estudo em relação a trabalhos anteriores ou tecnologias existentes. Se não encontrado, use null.
    19. `desafios_limitacoes_futuro`: (String) Desafios, limitações identificadas no estudo, ou sugestões para trabalhos futuros. Se não encontrado, use null.

    Texto para análise:
    ---------------------
    {text_content}
    ---------------------

    Retorne APENAS o objeto JSON. Certifique-se de que o JSON é válido.
    Se um campo não puder ser preenchido com base no texto, use `null` para strings/inteiros, ou uma lista vazia `[]` para listas.
    """
    return prompt

def extract_19_fields_with_gemini(text_content, retries=MAX_RETRIES, delay=RETRY_DELAY):
    if not model_19_fields:
        return {"error": "Modelo Gemini não inicializado."}
    if not text_content or pd.isna(text_content) or text_content.strip() == "":
        return {"error": "Conteúdo de texto de entrada vazio ou inválido."}

    prompt = generate_extraction_prompt(text_content)
    
    for attempt in range(retries):
        try:
            response = model_19_fields.generate_content(
                prompt,
                request_options={'timeout': REQUEST_TIMEOUT}
            )
            time.sleep(RATE_LIMIT_DELAY + random.uniform(0, 0.5)) # Respeitar limites da API

            if response.parts:
                content = response.text
                # Gemini pode retornar o JSON dentro de ```json ... ``` ou diretamente
                if content.startswith("```json"):
                    content = content[7:-3].strip()
                elif content.startswith("```JSON"):
                    content = content[7:-3].strip()
                elif content.startswith("```") and content.endswith("```"):
                     content = content[3:-3].strip()
                
                try:
                    parsed_json = json.loads(content)
                    # Validar se todos os 19 campos esperados estão presentes (mesmo que null/vazio)
                    missing_keys = [key for key in FIELDS_TO_EXTRACT if key not in parsed_json]
                    if missing_keys:
                        # Adicionar chaves ausentes com valor None para manter a estrutura consistente
                        for key in missing_keys:
                            parsed_json[key] = None # Ou [] se for uma lista, mas None é mais simples aqui
                        # print(f"Aviso: Chaves ausentes na resposta JSON foram preenchidas com None: {missing_keys}")
                    return parsed_json
                except json.JSONDecodeError as e:
                    error_msg = f"Erro ao decodificar JSON (tentativa {attempt + 1}/{retries}): {e}. Resposta: {content[:500]}..."
                    print(error_msg)
                    if attempt == retries - 1:
                        return {"error": error_msg, "raw_response": content}
                except Exception as e:
                    error_msg = f"Erro inesperado ao processar resposta (tentativa {attempt + 1}/{retries}): {e}. Resposta: {content[:500]}..."
                    print(error_msg)
                    if attempt == retries - 1:
                        return {"error": error_msg, "raw_response": content}
            else: # Resposta vazia ou bloqueada
                block_reason = response.prompt_feedback.block_reason if response.prompt_feedback else "Não especificado"
                error_msg = f"Resposta vazia ou bloqueada (tentativa {attempt + 1}/{retries}). Motivo: {block_reason}"
                print(error_msg)
                if attempt == retries - 1:
                    return {"error": error_msg}

        except genai.types.generation_types.BlockedPromptException as e:
            error_msg = f"Prompt bloqueado pela API (tentativa {attempt + 1}/{retries}): {e}"
            print(error_msg)
            if attempt == retries - 1:
                return {"error": error_msg}
        except Exception as e: # Outros erros (timeout, API key, rate limit)
            error_msg = f"Erro na chamada da API (tentativa {attempt + 1}/{retries}): {e}"
            print(error_msg)
            if "API key not valid" in str(e):
                 return {"error": "API Key inválida. Verifique sua chave."}
            if "429" in str(e) or "rate limit" in str(e).lower():
                print(f"Rate limit atingido. Aguardando {delay * 2}s.")
                time.sleep(delay * 2)
                delay *= 1.5 
            elif attempt == retries - 1:
                return {"error": error_msg}
        
        time.sleep(delay)
    
    return {"error": f"Falha ao obter extração da API Gemini após {retries} tentativas."}

# Teste rápido da função de prompt (opcional)
# sample_text = "O título é 'Super Nanotintas' por Dr. X em 2023. Usa TiO2 para UV block. Publicado no Journal of Coatings."
# print(generate_extraction_prompt(sample_text))

## 3. Processamento em Lote para Extração dos 19 Campos

In [34]:
# Limitar o número de registros para teste inicial, se desejado
NUM_SAMPLES_TO_PROCESS_19_FIELDS = config_sistema.get('NUM_SAMPLES_19_FIELDS_EXTRACTION', len(df_input)) # Processar todos por padrão

if df_input.empty or not api_key or not model_19_fields:
    print("Pré-requisitos não atendidos: DataFrame de entrada vazio, API Key não configurada ou modelo Gemini não inicializado. Pulando extração.")
    df_extracted_19_fields = pd.DataFrame() # DataFrame vazio
    # Criar colunas vazias para consistência do schema se o processamento for pulado
    empty_data = {key: [] for key in FIELDS_TO_EXTRACT}
    empty_data['error'] = []
    df_results_19_fields = pd.DataFrame(empty_data)

else:
    if 'Abstract_to_analyze' not in df_input.columns:
        print("Coluna 'Abstract_to_analyze' é crucial e não foi encontrada ou criada. A extração será de baixa qualidade.")
        # Tentar criar a coluna usando os nomes corretos das colunas
        if 'Article Title' in df_input.columns and 'Abstract' in df_input.columns:
            df_input['Abstract_to_analyze'] = df_input['Article Title'].fillna('') + " " + df_input['Abstract'].fillna('')
            print("Coluna 'Abstract_to_analyze' criada a partir da concatenação de 'Article Title' e 'Abstract'.")
        elif 'TI' in df_input.columns and 'AB' in df_input.columns:
            df_input['Abstract_to_analyze'] = df_input['TI'].fillna('') + " " + df_input['AB'].fillna('')
            print("Coluna 'Abstract_to_analyze' criada a partir da concatenação de 'TI' e 'AB'.")
        else:
            df_input['Abstract_to_analyze'] = ""

    df_to_process_19_fields = df_input.copy()
    if NUM_SAMPLES_TO_PROCESS_19_FIELDS < len(df_input):
        print(f"Processando uma amostra de {NUM_SAMPLES_TO_PROCESS_19_FIELDS} registros para extração dos 19 campos.")
        df_to_process_19_fields = df_input.sample(n=NUM_SAMPLES_TO_PROCESS_19_FIELDS, random_state=config_sistema.get('RANDOM_SEED', 42))
    else:
        print(f"Processando todos os {len(df_input)} registros para extração dos 19 campos.")
        # df_to_process_19_fields já é df_input.copy()

    # Aplicar a função de extração
    print(f"Iniciando extração dos 19 campos para {len(df_to_process_19_fields)} registros...")
    
    # Usar a coluna 'Abstract_to_analyze' que deve ter sido preparada no notebook 04 ou no início deste.
    # Se essa coluna estiver vazia ou for de má qualidade, a extração será ruim.
    results_19_fields = df_to_process_19_fields['Abstract_to_analyze'].progress_apply(extract_19_fields_with_gemini)
    
    # Converter os resultados (que são dicts ou dicts de erro) para um DataFrame
    df_results_19_fields = pd.json_normalize(results_19_fields)
    
    # Garantir que todas as 19 colunas de campos + coluna de erro existam, preenchendo com None se ausentes
    for col in FIELDS_TO_EXTRACT + ['error', 'raw_response']: # Adiciona raw_response para debug
        if col not in df_results_19_fields.columns:
            df_results_19_fields[col] = None
            
    # Juntar os resultados da extração com o DataFrame original (ou subset processado)
    # Certifique-se de que os índices estão alinhados para a junção
    df_extracted_19_fields = df_to_process_19_fields.join(df_results_19_fields.set_index(df_to_process_19_fields.index))
    
    print("\nExtração dos 19 campos concluída.")
    print(f"Colunas no DataFrame resultante: {df_extracted_19_fields.columns.tolist()}")
    print(df_extracted_19_fields.head())

    # Verificar a contagem de erros
    if 'error' in df_extracted_19_fields.columns:
        num_errors = df_extracted_19_fields['error'].notna().sum()
        print(f"Número de erros durante a extração: {num_errors} de {len(df_extracted_19_fields)}")
        if num_errors > 0:
            print("Distribuição dos erros:")
            print(df_extracted_19_fields['error'].value_counts())

Processando todos os 9046 registros para extração dos 19 campos.
Iniciando extração dos 19 campos para 9046 registros...


  0%|          | 0/9046 [00:00<?, ?it/s]

Erro na chamada da API (tentativa 1/3): 404 models/gemini-2.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro na chamada da API (tentativa 2/3): 404 models/gemini-2.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro na chamada da API (tentativa 2/3): 404 models/gemini-2.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro na chamada da API (tentativa 3/3): 404 models/gemini-2.0-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Erro na chamada da API (tentativa 3/3): 404 models/gemini-2.0-pro is not found for API version v1bet

KeyboardInterrupt: 

## 4. Salvamento dos Resultados Detalhados e Sumarização

In [ ]:
extraction_summary_19_fields = {}

if not df_extracted_19_fields.empty:
    # Salvar o DataFrame com os 19 campos extraídos
    df_extracted_19_fields.to_csv(EXTRACTED_19_FIELDS_OUTPUT_FILE, index=False, encoding='utf-8')
    print(f"Dataset com 19 campos extraídos salvo em: {EXTRACTED_19_FIELDS_OUTPUT_FILE}")

    # Gerar estatísticas/sumário da extração
    total_records_attempted = len(df_extracted_19_fields)
    successful_extractions = 0
    failed_extractions = 0
    error_messages_distribution = {}

    if 'error' in df_extracted_19_fields.columns:
        successful_extractions = df_extracted_19_fields['error'].isna().sum()
        failed_extractions = df_extracted_19_fields['error'].notna().sum()
        if failed_extractions > 0:
            error_messages_distribution = df_extracted_19_fields.loc[df_extracted_19_fields['error'].notna(), 'error'].value_counts().to_dict()
    else: # Caso a coluna 'error' não exista (improvável com a lógica acima, mas para segurança)
        if api_key and model_19_fields and not df_input.empty : # Se a extração deveria ter rodado
             successful_extractions = total_records_attempted 
    
    # Contagem de preenchimento para cada um dos 19 campos
    field_fill_counts = {}
    for field in FIELDS_TO_EXTRACT:
        if field in df_extracted_19_fields.columns:
            # Conta não nulos e não vazios (para listas/strings)
            non_empty_count = df_extracted_19_fields[field].apply(lambda x: False if (x is None or (isinstance(x, (str, list)) and not x)) else True).sum()
            field_fill_counts[field] = {
                "filled": int(non_empty_count),
                "total": int(total_records_attempted),
                "fill_rate_percent": round((non_empty_count / total_records_attempted) * 100, 2) if total_records_attempted > 0 else 0
            }
        else:
            field_fill_counts[field] = {"filled": 0, "total": int(total_records_attempted), "fill_rate_percent": 0}


    extraction_summary_19_fields = {
        "total_records_attempted_extraction": total_records_attempted,
        "successful_full_json_extractions": int(successful_extractions), # JSON decodificado sem erro fatal
        "failed_full_json_extractions": int(failed_extractions),
        "error_distribution": error_messages_distribution,
        "field_fill_statistics": field_fill_counts,
        "output_file_19_fields": str(EXTRACTED_19_FIELDS_OUTPUT_FILE)
    }

    with open(EXTRACTED_19_FIELDS_SUMMARY_FILE, 'w', encoding='utf-8') as f:
        json.dump(extraction_summary_19_fields, f, ensure_ascii=False, indent=4)
    print(f"Sumário da extração dos 19 campos salvo em: {EXTRACTED_19_FIELDS_SUMMARY_FILE}")
    print("\nSumário da Extração Detalhada:")
    print(json.dumps(extraction_summary_19_fields, indent=2, ensure_ascii=False))
else:
    print("Nenhum resultado da extração dos 19 campos para salvar ou sumarizar.")
    extraction_summary_19_fields = {
        "total_records_attempted_extraction": 0,
        "successful_full_json_extractions": 0,
        "failed_full_json_extractions": 0,
        "error_distribution": {},
        "field_fill_statistics": {field: {"filled": 0, "total": 0, "fill_rate_percent": 0} for field in FIELDS_TO_EXTRACT},
        "message": "Extração dos 19 campos não executada ou não produziu resultados."
    }
    with open(EXTRACTED_19_FIELDS_SUMMARY_FILE, 'w', encoding='utf-8') as f:
        json.dump(extraction_summary_19_fields, f, ensure_ascii=False, indent=4)
    print(f"Arquivo de sumário da extração (vazio) salvo em: {EXTRACTED_19_FIELDS_SUMMARY_FILE}")

Dataset com 19 campos extraídos salvo em: ../data/processed/dataset_with_19_fields.csv
Sumário da extração dos 19 campos salvo em: ../data/processed/extraction_19_fields_summary.json

Sumário da Extração Detalhada:
{
  "total_records_attempted_extraction": 9046,
  "successful_full_json_extractions": 0,
  "failed_full_json_extractions": 9046,
  "error_distribution": {
    "Conteúdo de texto de entrada vazio ou inválido.": 9046
  },
  "field_fill_statistics": {
    "id_documento": {
      "filled": 0,
      "total": 9046,
      "fill_rate_percent": 0.0
    },
    "titulo": {
      "filled": 0,
      "total": 9046,
      "fill_rate_percent": 0.0
    },
    "ano": {
      "filled": 0,
      "total": 9046,
      "fill_rate_percent": 0.0
    },
    "tipo_documento": {
      "filled": 0,
      "total": 9046,
      "fill_rate_percent": 0.0
    },
    "autores": {
      "filled": 0,
      "total": 9046,
      "fill_rate_percent": 0.0
    },
    "afiliacoes_autores": {
      "filled": 0,
      "

## 5. Atualização do Arquivo de Configuração do Sistema

In [ ]:
if config_path.exists():
    with open(config_path, 'r', encoding='utf-8') as f:
        config_sistema = json.load(f)
else:
    config_sistema = {} # Deveria existir, mas como fallback

config_sistema['EXTRACTION_19_FIELDS_COMPLETED'] = True
config_sistema['EXTRACTION_19_FIELDS_OUTPUT_FILE'] = str(EXTRACTED_19_FIELDS_OUTPUT_FILE)
config_sistema['EXTRACTION_19_FIELDS_SUMMARY_FILE'] = str(EXTRACTED_19_FIELDS_SUMMARY_FILE)
config_sistema['TIMESTAMP_EXTRACTION_19_FIELDS'] = time.strftime('%Y-%m-%d %H:%M:%S')

# Adicionar alguns parâmetros usados nesta etapa para rastreabilidade
config_sistema['PARAMS_19_FIELDS_EXTRACTION'] = {
    'GEMINI_MODEL_NAME': GEMINI_MODEL_NAME,
    'MAX_RETRIES': MAX_RETRIES,
    'RETRY_DELAY': RETRY_DELAY, # O valor inicial, pode aumentar dinamicamente
    'REQUEST_TIMEOUT': REQUEST_TIMEOUT,
    'RATE_LIMIT_DELAY': RATE_LIMIT_DELAY,
    'NUM_SAMPLES_PROCESSED': NUM_SAMPLES_TO_PROCESS_19_FIELDS if not df_extracted_19_fields.empty else 0,
    'FIELDS_TARGETED': FIELDS_TO_EXTRACT
}


with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config_sistema, f, indent=4, ensure_ascii=False)

print(f"Arquivo de configuração '{config_path}' atualizado com o status da extração dos 19 campos.")

Arquivo de configuração 'config_sistema.json' atualizado com o status da extração dos 19 campos.


## Próximos Passos

Com a extração detalhada dos 19 campos concluída, o dataset está significativamente enriquecido. Os próximos passos podem incluir:

1.  **Validação e Limpeza dos Dados Extraídos:** Uma análise mais aprofundada da qualidade dos dados extraídos, seguida de limpeza e normalização, se necessário.
2.  **Análise Exploratória dos Dados (EDA):** Investigar as distribuições, correlações e padrões nos 19 campos.
3.  **Engenharia de Features:** Criação de novas features a partir dos campos extraídos para modelos de machine learning.
4.  **Desenvolvimento de Modelos Preditivos:** Utilizar os dados para treinar modelos que possam prever, por exemplo, propriedades de tintas ou adequação de nanomateriais.
5.  **Construção de um Sistema de Busca e Recomendação:** Usar os dados estruturados para permitir buscas complexas e recomendar artigos/patentes ou formulações.
6.  **Visualização e Dashboarding:** Criar dashboards para explorar os insights gerados.